# Spark Session

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         #.config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .config("spark.driver.extraClassPath", "/usr/local/spark/resources/jars/postgresql-9.4.1207.jar")
         .getOrCreate())
sc = spark.sparkContext

21/12/29 16:16:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.sql import SparkSession

# Spark session & context
spark_clt = (SparkSession
         .builder
         .master("spark://spark:7077")
         .appName("read-postgres")
         # Add postgres jar
         #.config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .config("spark.driver.extraClassPath", "/usr/local/spark/resources/jars/postgresql-9.4.1207.jar")
         #.enableHiveSupport()
         .getOrCreate())

In [4]:
spark_clt.sql("show databases").show()

21/12/29 04:10:39 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/29 04:10:39 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/12/29 04:10:43 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/12/29 04:10:43 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.21.0.4
21/12/29 04:10:43 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


+---------+
|namespace|
+---------+
|  default|
+---------+



In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [2]:
df_movies = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/airflow")
    .option("dbtable", "public.dag")
    .option("user", "airflow")
    .option("password", "airflow")
    .load() )

df_movies.show()

+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|              dag_id|is_paused|is_subdag|is_active|  last_scheduler_run|last_pickled|last_expired|scheduler_lock|pickle_id|             fileloc| owners|         description|default_view|   schedule_interval|root_dag_id|
+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|spark-hello-world...|     true|    false|     true|2021-12-29 16:16:...|        null|        null|          null|     null|/usr/local/airflo...|airflow|This DAG runs a P...|        null|{"type": "timedel...|       null|
|          spark-test|     true|    false|     true|2021-12-29 16:16:...|        null|        null|          null|  

In [4]:
df_movies = (
    spark_clt.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/airflow")
    .option("dbtable", "public.dag")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

In [5]:
df_movies.show()

+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|              dag_id|is_paused|is_subdag|is_active|  last_scheduler_run|last_pickled|last_expired|scheduler_lock|pickle_id|             fileloc| owners|         description|default_view|   schedule_interval|root_dag_id|
+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|spark-hello-world...|     true|    false|     true|2021-12-29 16:17:...|        null|        null|          null|     null|/usr/local/airflo...|airflow|This DAG runs a P...|        null|{"type": "timedel...|       null|
|          spark-test|     true|    false|     true|2021-12-29 16:17:...|        null|        null|          null|  

# Read Postgres

In [3]:
df_movies = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/test")
    .option("dbtable", "public.movies")
    .option("user", "test")
    .option("password", "postgres")
    .load()
)

In [5]:
df_ratings = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/test")
    .option("dbtable", "public.ratings")
    .option("user", "test")
    .option("password", "postgres")
    .load()
)

# Top 10 movies with more ratings

In [10]:
df_movies = df_movies.alias("m")
df_ratings = df_ratings.alias("r")

df_join = df_ratings.join(df_movies, df_ratings.movieId == df_movies.movieId).select("r.*","m.title")

In [22]:
from pyspark.sql import functions as F

df_result = (
    df_join
    .groupBy("title")
    .agg(
        F.count("timestamp").alias("qty_ratings")
        ,F.mean("rating").alias("avg_rating")
    )
    .sort(F.desc("qty_ratings"))
    .limit(10)
)

In [24]:
df_result.coalesce(1).write.format("csv").mode("overwrite").save("/home/jovyan/work/data/output_postgres", header=True)

In [14]:
# Spark Cluster

In [15]:
df_movies = (
    spark_clt.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/test")
    .option("dbtable", "public.movies")
    .option("user", "test")
    .option("password", "postgres")
    .load()
)

In [16]:
df_movies.registerTempTable("test")

In [17]:
spark_clt.sql("create table test_table as ( select * from test)")

21/12/26 16:08:52 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `default`.`test_table`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists
+- Project [movieId#27, title#28, genres#29]
   +- SubqueryAlias test
      +- Relation[movieId#27,title#28,genres#29] JDBCRelation(public.movies) [numPartitions=1]
